In [1]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2024-07-16 23:17:15.292840: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 23:17:15.292945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 23:17:15.425217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [6]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on chess dataset', 
    job_type="training", 
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: krishnaharshith1234 (krishnaharshith1234-bits-pilani). Use `wandb login --relogin` to force relogin


In [7]:
base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
dataset_name = "/kaggle/input/finetuningchess/Fine_tuning_Dataset.csv"
new_model = "llama-3-8b-chess-chat"

In [8]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [9]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [27]:
import pandas as pd
from datasets import Dataset

# Load the uploaded dataset
file_path = '/kaggle/input/finetuningchess/Fine_tuning_Dataset.csv'
df = pd.read_csv(file_path)

# Convert the pandas DataFrame to a Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Select a subset of the dataset for demonstration purposes (e.g., first 40 samples)
dataset = dataset.select(range(48))

# Define the function to format the chat template
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["User"]},
                {"role": "assistant", "content": row["Output"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Apply the formatting function to the dataset
dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

# Display the formatted text of the fourth sample
print(dataset['text'][3])


/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/48 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, a

<|im_start|>user
FEN_INPUT = r5rk/3nbp1p/pp1p1n2/7B/2q1PB2/P1N4R/1PQ5/1K5R b - - 1 1,
    PGN_INPUT = [Variant "From Position"]
[FEN "r5rk/3nbp1p/pp1p1n2/7B/2q1PB2/P1N4R/1PQ5/1K1R4 w - - 0 1"]

1. Rdh1,
    BEST_MOVE_INPUT = g8g7,
    SEQUENCE_OF_BEST_MOVES_INPUT = ['g8g7', 'f4h6', 'g7g6', 'h5g6', 'f7g6', 'h6g5', 'a8g8', 'c3d5', 'c4c2', 'b1c2']
    ALTERNATIVE_MOVE_1 = h8g7
    ALTERNATIVE_MOVE_2 = c4d4
    Explain why it is the best move, with a concise yet comprehensive explanation in points<|im_end|>
<|im_start|>assistant
Here's a comprehensive analysis of the best move g8g7:

**Development and Control:**

* g8g7 develops the kingside pieces, preparing to connect the rooks and potentially create a strong pawn center.
* The move also aims to control key squares on the kingside, such as g6 and f6, which can become important outposts for the pieces.

**Coordination:**

* g8g7 improves the coordination between the king, queen, and rooks, allowing for potential attacks on the kingside an

In [28]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load the uploaded dataset
file_path = '/kaggle/input/finetuningchess/Fine_tuning_Dataset.csv'
df = pd.read_csv(file_path)

# Drop the unnecessary column
df = df.drop(columns=["Unnamed: 0"])

# Convert the pandas DataFrame to a Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into training and test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
test_dataset = dataset['test']

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define the function to format the chat template
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["User"]},
                {"role": "assistant", "content": row["Output"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Apply the formatting function to the training and test sets
train_dataset = train_dataset.map(format_chat_template, num_proc=4)
test_dataset = test_dataset.map(format_chat_template, num_proc=4)

# Display the formatted text of the fourth sample in the training set
print(train_dataset['text'][3])

# Display the formatted text of the fourth sample in the test set
print(test_dataset['text'][3])


/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/40 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, a

Map (num_proc=4):   0%|          | 0/10 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
No chat template is set for this tokenizer, falling back to a default clas

<|im_start|>user
FEN_INPUT = 4q1k1/p5pp/2p5/p2p4/P2Pn3/1Pb4P/4QPP1/2B3K1 w - - 0 5,
    PGN_INPUT = [Variant "From Position"]
[FEN "3qr1k1/p5pp/2pb1n2/p2p4/P7/1PNP3P/3Q1PP1/2B1R1K1 b - - 0 1"]

1... Bb4 2. Rxe8+ Qxe8 3. Bb2 (3. d4 Ne4 4. Qe2 Bxc3),
    BEST_MOVE_INPUT = c1e3,
    SEQUENCE_OF_BEST_MOVES_INPUT = ['c1e3', 'e4d6', 'e2a6', 'c3b4', 'g2g4', 'h7h5', 'g1h2', 'd6e4', 'a6d3', 'e8g6']
    ALTERNATIVE_MOVE_1 = e2d3
    ALTERNATIVE_MOVE_2 = e2d1
    Explain why it is the best move, with a concise yet comprehensive explanation in points<|im_end|>
<|im_start|>assistant
Here's a comprehensive analysis of the best move c1e3:

**Development and Control:**

* c1e3 develops the bishop, putting it on a more active square and controlling the e-file.
* It also prepares to develop other pieces, such as the queen and knight, by opening up the e-file and d-file.
* Controlling the center with the bishop on e3 helps to restrict Black's mobility and creates potential weaknesses on the d-file.

**Co

In [29]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
from trl import SFTConfig, SFTTrainer



# Setup the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [31]:
trainer.train()

Step,Training Loss,Validation Loss
4,3.816500,3.835463
8,3.485600,3.172841
12,2.860600,2.664764
16,2.227100,2.323996
20,2.126600,2.156408


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /kaggle/input/llama-3/transformers/8b-chat-hf/1 - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=20, training_loss=3.039326798915863, metrics={'train_runtime': 182.5777, 'train_samples_per_second': 0.219, 'train_steps_per_second': 0.11, 'total_flos': 927360111083520.0, 'train_loss': 3.039326798915863, 'epoch': 1.0})

In [32]:
wandb.finish()
model.config.use_cache = True

eval/loss,█▆▅▄▄▃▂▂▁▁
eval/runtime,▁▁▁▇▇▂██▇▇
eval/samples_per_second,███▁▁▅▁▁▁▁
eval/steps_per_second,███▁▁▅▁▁▁▁
train/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▇▇▇███▁▁▂▂▂▃▄▄▄▄▅▅▅▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▇▇▇███▁▁▂▂▂▃▄▄▄▄▅▅▅▆▇▇▇███
train/grad_norm,▁▁▁▁▁▂▃▅▆▅▄▆▄▃▃▂▂▃▂▂▂▂▁▂▄▆▄█▅▅▇▅▄▆▅▆▅▄▄▃
train/learning_rate,▂▂▃▄▅▅▆▇▇█▇▇▆▅▅▄▃▂▂▁▂▂▃▄▅▅▆▇▇█▇▇▆▅▅▄▃▂▂▁
train/loss,████▇▇▇▇▆▅▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
eval/loss,2.15641
eval/runtime,13.7674


In [36]:
messages = [
    {
        "role": "user",
        "content": """
        FEN_INPUT = 4r1k1/1pqnrpp1/p4n1p/P1pP4/R2bp3/1PN3P1/2PB1P1P/3QRBK1 w - - 1 2,
        PGN_INPUT = [Variant From Position]
        [FEN "4r1k1/1pqnrpp1/p4n1p/P1pPb3/R3p3/1PN3P1/2PB1P1P/3QRBK1 b - - 0 1"]

        Bd4,
        BEST_MOVE_INPUT = f1g2,
        SEQUENCE_OF_BEST_MOVES_INPUT = ['f1g2', 'c7c8', 'e1e2', 'e7e5', 'd1e1', 'b7b5', 'a5b6', 'd7b6', 'a4a2', 'f6d5']
        ALTERNATIVE_MOVE_1 = d2e3
        ALTERNATIVE_MOVE_2 = f1h3
        Explain why it is the best move, with a concise yet comprehensive explanation in points
        """
    }
]

# Apply the chat template
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Prepare the inputs for the model
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

# Generate the output
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=512, num_return_sequences=1)

# Decode the generated text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated response
print(text.split("assistant")[1])

 here's a comprehensive analysis of the best move ` : * * development and control : * * * ` : * * * * * coordination : * * * * ` : * * * * * * tactical insights : * * * * * * * ` : * * * * * * * * tactical and the center of the move ` : * * * * * * * * * tactical and the center of the move ` c7c8 ` : * * * * * * * * * * tactical and the center of the move ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 ` c7c8 `
